In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import lenin
import mnist

RANDOM_SEED = 190119

In [2]:
X_train, y_train, X_test, y_test = mnist.load()

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100.0%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Save complete.


In [3]:
num_labels = len(y_train)
num_labels

60000

In [4]:
# Будем писать простой классификатор
# Для этого выходные данные из чисел 0,1,2,3,..,9
# Нужно преобразовать к выходному слою классификатора
# [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] - 0
# [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0] - 5
# [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1] - 9

# Данными для обучения
# Заполняем все нулями
num_labels = len(y_train)
train_labels = np.zeros((num_labels, 10))
# Ставим единички, в соответсвии с числом
for i in range(num_labels):
    train_labels[i][y_train[i]] = 1

# Данные для тестирования
# Заполняем все нулями
num_labels = len(y_test)
test_labels = np.zeros((num_labels, 10))
# Ставим единички, в соответсвии с числом
for i in range(num_labels):
    test_labels[i][y_test[i]] = 1

In [5]:
X_train, X_test = X_train - np.mean(X_train), X_test - np.mean(X_train)

In [7]:
np.min(X_train), np.max(X_train), np.min(X_test), np.max(X_test)

(np.float64(-33.318421449829934),
 np.float64(221.68157855017006),
 np.float64(-33.318421449829934),
 np.float64(221.68157855017006))

In [8]:
X_train, X_test = X_train / np.std(X_train), X_test / np.std(X_train)

In [9]:
np.min(X_train), np.max(X_train), np.min(X_test), np.max(X_test)

(np.float64(-0.424073894391566),
 np.float64(2.821543345689335),
 np.float64(-0.424073894391566),
 np.float64(2.821543345689335))

In [10]:
# Tanh и MSE
model = lenin.NeuralNetwork(
    layers=[
        lenin.Dense(neurons=89, activation=lenin.Tanh()),
        lenin.Dense(neurons=10, activation=lenin.Tanh()),
    ],
    loss=lenin.MeanSquaredError(normalize=False),
    seed=RANDOM_SEED,
)

trainer = lenin.Trainer(model, lenin.SGD(0.1))
trainer.fit(
    X_train,
    train_labels,
    X_test,
    test_labels,
    epochs=50,
    eval_every=10,
    seed=RANDOM_SEED,
    batch_size=60,
)

lenin.mmetric.calc_accuracy_model(trainer.net, X_test, y_test)

[ 10 epoch] loss: 2.960 | accuracy: 49.02%
[ 20 epoch] loss: 2.128 | accuracy: 56.25%
[ 30 epoch] loss: 1.545 | accuracy: 66.02%
[ 40 epoch] loss: 1.419 | accuracy: 67.66%
[ 50 epoch] loss: 1.324 | accuracy: 79.91%


'The model validation accuracy is: 79.91%'

In [ ]:
# MSE и нормализация данных при вычислении ошибки
model = lenin.NeuralNetwork(
    layers=[
        lenin.Dense(neurons=89, activation=lenin.Sigmoid()),
        lenin.Dense(neurons=10, activation=lenin.Sigmoid()),
    ],
    loss=lenin.MeanSquaredError(normalize=True),
    seed=RANDOM_SEED,
)

trainer = lenin.Trainer(model, lenin.SGD(0.1))
trainer.fit(
    X_train,
    train_labels,
    X_test,
    test_labels,
    epochs=50,
    eval_every=10,
    seed=RANDOM_SEED,
    batch_size=60,
)

lenin.mmetric.calc_accuracy_model(trainer.net, X_test, y_test)

[ 10] loss: 0.576 | accuracy: 62.54%
[ 20] loss: 0.577 | accuracy: 62.32%

Loss increased after epoch: 20
using model[ 10] loss:0.576


'The model validation accuracy is: 62.54%'

In [ ]:
# SoftmaxCrossEntropy
model = lenin.NeuralNetwork(
    layers=[
        lenin.Dense(neurons=89, activation=lenin.Tanh()),
        lenin.Dense(neurons=10, activation=lenin.Linear()),
    ],
    loss=lenin.SoftmaxCrossEntropy(),
    seed=RANDOM_SEED,
)

trainer = lenin.Trainer(model, lenin.SGD(0.1))
trainer.fit(
    X_train,
    train_labels,
    X_test,
    test_labels,
    epochs=50,
    eval_every=10,
    seed=RANDOM_SEED,
    batch_size=60,
)

lenin.mmetric.calc_accuracy_model(trainer.net, X_test, y_test)

[ 10] loss: 0.637 | accuracy: 89.29%
[ 20] loss: 0.578 | accuracy: 90.43%
[ 30] loss: 0.557 | accuracy: 90.73%
[ 40] loss: 0.555 | accuracy: 91.08%
[ 50] loss: 0.556 | accuracy: 91.41%

Loss increased after epoch: 50
using model[ 40] loss:0.555


'The model validation accuracy is: 91.08%'

In [ ]:
# SoftmaxCrossEntropy + SGDMomentum
model = lenin.NeuralNetwork(
    layers=[
        lenin.Dense(neurons=89, activation=lenin.Tanh()),
        lenin.Dense(neurons=10, activation=lenin.Linear()),
    ],
    loss=lenin.SoftmaxCrossEntropy(),
    seed=RANDOM_SEED,
)

trainer = lenin.Trainer(model, lenin.SGDMomentum(lr=0.1, momentum=0.9))
trainer.fit(
    X_train,
    train_labels,
    X_test,
    test_labels,
    epochs=50,
    eval_every=10,
    seed=RANDOM_SEED,
    batch_size=60,
)

lenin.mmetric.calc_accuracy_model(trainer.net, X_test, y_test)

[ 10] loss: 0.402 | accuracy: 93.56%
[ 20] loss: 0.362 | accuracy: 94.53%
[ 30] loss: 0.330 | accuracy: 95.18%
[ 40] loss: 0.369 | accuracy: 94.91%

Loss increased after epoch: 40
using model[ 30] loss:0.330


'The model validation accuracy is: 95.18%'

In [ ]:
# SoftmaxCrossEntropy + SGDMomentum + DecayLinear
model = lenin.NeuralNetwork(
    layers=[
        lenin.Dense(neurons=89, activation=lenin.Tanh()),
        lenin.Dense(neurons=10, activation=lenin.Linear()),
    ],
    loss=lenin.SoftmaxCrossEntropy(),
    seed=RANDOM_SEED,
)

trainer = lenin.Trainer(model,
                        lenin.SGDMomentum(lr=0.15,
                                          final_lr=0.05,
                                          decay_type="linear",
                                          momentum=0.9))
trainer.fit(
    X_train,
    train_labels,
    X_test,
    test_labels,
    epochs=50,
    eval_every=10,
    seed=RANDOM_SEED,
    batch_size=60,
)

lenin.mmetric.calc_accuracy_model(trainer.net, X_test, y_test)

[ 10] loss: 0.375 | accuracy: 94.10%
[ 20] loss: 0.391 | accuracy: 93.97%

Loss increased after epoch: 20
using model[ 10] loss:0.375


'The model validation accuracy is: 94.10%'

In [ ]:
# SoftmaxCrossEntropy + SGDMomentum + DecayExponential
model = lenin.NeuralNetwork(
    layers=[
        lenin.Dense(neurons=89, activation=lenin.Tanh()),
        lenin.Dense(neurons=10, activation=lenin.Linear()),
    ],
    loss=lenin.SoftmaxCrossEntropy(),
    seed=RANDOM_SEED,
)

trainer = lenin.Trainer(model,
                        lenin.SGDMomentum(lr=0.2,
                                          final_lr=0.05,
                                          decay_type="exponential",
                                          momentum=0.9))
trainer.fit(
    X_train,
    train_labels,
    X_test,
    test_labels,
    epochs=50,
    eval_every=10,
    seed=RANDOM_SEED,
    batch_size=60,
)

lenin.mmetric.calc_accuracy_model(trainer.net, X_test, y_test)

[ 10] loss: 0.467 | accuracy: 93.04%
[ 20] loss: 0.311 | accuracy: 95.49%
[ 30] loss: 0.290 | accuracy: 95.76%
[ 40] loss: 0.306 | accuracy: 96.02%

Loss increased after epoch: 40
using model[ 30] loss:0.290


'The model validation accuracy is: 95.76%'

In [ ]:
# SoftmaxCrossEntropy + SGDMomentum + DecayExponential + WeightGLorotInit
model = lenin.NeuralNetwork(
    layers=[
        lenin.Dense(neurons=89, activation=lenin.Tanh(), weight_init="glorot"),
        lenin.Dense(neurons=10, activation=lenin.Linear(),
                    weight_init="glorot")
    ],
    loss=lenin.SoftmaxCrossEntropy(),
    seed=RANDOM_SEED,
)

trainer = lenin.Trainer(model,
                        lenin.SGDMomentum(lr=0.2,
                                          final_lr=0.05,
                                          decay_type="exponential",
                                          momentum=0.9))
trainer.fit(
    X_train,
    train_labels,
    X_test,
    test_labels,
    epochs=50,
    eval_every=10,
    seed=RANDOM_SEED,
    batch_size=60,
)

lenin.mmetric.calc_accuracy_model(trainer.net, X_test, y_test)

[ 10] loss: 0.344 | accuracy: 94.74%
[ 20] loss: 0.272 | accuracy: 95.86%
[ 30] loss: 0.254 | accuracy: 96.54%
[ 40] loss: 0.275 | accuracy: 96.26%

Loss increased after epoch: 40
using model[ 30] loss: 0.254


'The model validation accuracy is: 96.54%'

In [ ]:
# SoftmaxCrossEntropy + SGDMomentum + DecayExponential + WeightGLorotInit + Dropout
model = lenin.NeuralNetwork(
    layers=[
        lenin.Dense(neurons=89,
                    activation=lenin.Tanh(),
                    weight_init="glorot",
                    dropout=0.8),
        lenin.Dense(neurons=10, activation=lenin.Linear(),
                    weight_init="glorot")
    ],
    loss=lenin.SoftmaxCrossEntropy(),
    seed=RANDOM_SEED,
)

trainer = lenin.Trainer(model,
                        lenin.SGDMomentum(lr=0.2,
                                          final_lr=0.05,
                                          decay_type="exponential",
                                          momentum=0.9))
trainer.fit(
    X_train,
    train_labels,
    X_test,
    test_labels,
    epochs=50,
    eval_every=10,
    seed=RANDOM_SEED,
    batch_size=60,
)

lenin.mmetric.calc_accuracy_model(trainer.net, X_test, y_test)

[ 10] loss: 0.281 | accuracy: 95.17%
[ 20] loss: 0.225 | accuracy: 96.27%
[ 30] loss: 0.209 | accuracy: 96.56%
[ 40] loss: 0.195 | accuracy: 96.85%
[ 50] loss: 0.188 | accuracy: 96.82%


'The model validation accuracy is: 96.82%'

In [12]:
# DeepNet + SoftmaxCrossEntropy + SGDMomentum + DecayExponential + WeightGLorotInit + Dropout
model = lenin.NeuralNetwork(
    layers=[
        lenin.Dense(neurons=178,
                    activation=lenin.Tanh(),
                    weight_init="glorot",
                    dropout=0.8),
        lenin.Dense(neurons=46,
                    activation=lenin.Tanh(),
                    weight_init="glorot",
                    dropout=0.8),
        lenin.Dense(neurons=10,
                    activation=lenin.Linear(),
                    weight_init="glorot")
    ],
    loss=lenin.SoftmaxCrossEntropy(),
    seed=RANDOM_SEED,
)

trainer = lenin.Trainer(model,
                        lenin.SGDMomentum(lr=0.2,
                                          final_lr=0.05,
                                          decay_type="exponential",
                                          momentum=0.9))
trainer.fit(
    X_train,
    train_labels,
    X_test,
    test_labels,
    epochs=50,
    eval_every=10,
    seed=RANDOM_SEED,
    batch_size=60,
)

lenin.mmetric.calc_accuracy_model(trainer.net, X_test, y_test)

[ 10 epoch] loss: 0.297 | accuracy: 95.14%
[ 20 epoch] loss: 0.251 | accuracy: 95.91%
[ 30 epoch] loss: 0.207 | accuracy: 96.66%
[ 40 epoch] loss: 0.202 | accuracy: 96.86%
[ 50 epoch] loss: 0.181 | accuracy: 97.09%


'The model validation accuracy is: 97.09%'

In [9]:
X_train_conv, X_test_conv = X_train.reshape(-1, 1, 28, 28), X_test.reshape(-1, 1, 28, 28)

In [ ]:
model = lenin.NeuralNetwork(
    layers=[lenin.Conv2D(out_channels=32,
                         param_size=5,
                         dropout=0.8,
                         weight_init="glorot",
                         activation=lenin.Tanh(),
                         flatten=True,
                         fast=True),
            lenin.Dense(neurons=10,
                        activation=lenin.Linear())],
    loss=lenin.SoftmaxCrossEntropy(),
    seed=20190402)

trainer = lenin.Trainer(model, lenin.SGDMomentum(lr=0.1, momentum=0.9))
trainer.fit(X_train_conv, train_labels, X_test_conv, test_labels,
            epochs=1,
            eval_every=1,
            seed=20190402,
            batch_size=60,
            conv_testing=True)